In [1]:
cd /data/projects/windturbine-identification/MachineLearningCourse

/data/projects/windturbine-identification/MachineLearningCourse


In [2]:
import ee
from pathlib import Path
import os
import pandas as pd
import numpy as np

ee.Initialize()


def read_params():
    
    config_file = "config"
    
    
    files = [x for x in os.listdir(config_file) if (x.endswith(".csv") and x.startswith("params"))]
    p = {}
    
    countries = []
    
    for i in files:
        country = i[6:-4]
        countries.append(country)
        
        p[country] = pd.read_csv(config_file + "/" + i)
    
    return((countries,p))
        
COUNTRIES, PARAMS = read_params()


def get_param(country,name):
    val=PARAMS[country].at[0,name]
    return(val)

COUNTRY = 'AT'

# Test the earthengine command by getting help on upload.
#!earthengine upload image -h

#%config IPCompleter.greedy=True




In [5]:
turbines = pd.read_csv(get_param(COUNTRY, "FILE_TURBINE_LOCATIONS"))

print(turbines.shape)

idx = turbines['Jahr'] > 1999

turbines = turbines[idx]

print(turbines.shape)

n = turbines.shape[0]
results = np.zeros((n, 2))

turbines = turbines.drop_duplicates(['Lat', 'Long'])
print(turbines.shape)

#turbines_1 = turbines
#turbines_1['Lat'] = turbines_1['Lat'].round(5)
#turbines_1['Long'] = turbines_1['Long'].round(5)

#turbines_1 = turbines_1.drop_duplicates(['Lat', 'Long'])

#print(turbines_1.shape)

#turbines = turbines_1
pts = []
for i in range(0, turbines.shape[0]):
#for i in range(0, 550):

#for i in range(0, 10):

    x = turbines.iloc[i]["Long"]
    y = turbines.iloc[i]["Lat"] 
    
    p = ee.Feature(ee.Geometry.Point(x, y))
    pts.append(p)
    
print(len(pts))
    
pts = ee.List(pts)
   
pts = ee.FeatureCollection(pts)

(1223, 15)
(1088, 15)
(1085, 15)
1085


In [6]:
pts.getInfo()

{'type': 'FeatureCollection',
 'columns': {'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [16.42768, 48.50219]},
   'id': '0',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [16.43274, 48.50329]},
   'id': '1',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [16.42746, 48.5045]},
   'id': '2',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [16.42726, 48.507]},
   'id': '3',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [16.42729, 48.50999]},
   'id': '4',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [16.42709, 48.51308]},
   'id': '5',
   'properties': {}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [16.427670000000003, 48.49985]},
   'id': '6',
   'properties': {}},
  {'

In [7]:
def one_year(image, l):

#    image = images[year]
    #print("test")
    
    bands_select = [f'{band}_{reducer_str}' for band in bands]

    colReducer = ee.Reducer.toList()

    pixels = image.select(bands_select).reduceRegion(colReducer, pts, 30)

    #data = [pixels.get(k) for k in bands_select]
    
    #d = [v.getInfo() for v in pixels]
    
    # Convert to Number for further use
    #dataN = [ee.Number(v) for v in data]
    
    #d = [v.getInfo() for v in dataN]

    #print(d)
    #d = [v.getInfo() for v in data]

    #numpy_vals = np.array(d).transpose()
    
    l = ee.List(l).add(pixels)

    return(l)    
    


        




In [8]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
#bands = ['B2', 'B3', 'B4']
# Use Landsat 8 surface reflectance data.
l8sr = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")

# Cloud masking function.
def maskL8sr(image):
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask).select(bands).divide(10000)

# The image input data is a 2018 cloud-masked median composite.
reducer = ee.Reducer.median()

reducer_str = "median"

#years = range(2000, 2019)


    


In [10]:

def result_pixels(years, l8ssr):
    images = []

    for year in years:
        image = l8sr.filterDate(str(year) + '-01-01', str(year) + '-12-31').map(maskL8sr).reduce(reducer)
        images.append(image)
    
    image_collection = ee.ImageCollection(images)
    
    
    
    
    l = ee.List([])

    result = ee.List(image_collection.iterate(one_year, l))

    ee.batch.Export.image.toDrive({'collection': result, 'description': 'windturbine pixels'})
    
    return(1)
    #ar = result.getInfo()
    
    #return(ar)


In [ ]:
t = [[[37.24154695, 8.01665693],
          [37.26876315, 8.01665693],
          [37.26876315, 8.04378808],
          [37.24154695, 8.04378808],
          [37.24154695, 8.01665693]],
         [[37.34154695, 8.11665693],
          [37.36876315, 8.11665693],
          [37.36876315, 8.14378808],
          [37.34154695, 8.14378808],
          [37.34154695, 8.11665693]]]

region = ee.Geometry.Polygon(t)
dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(region) \
        .filterDate('2016-01-01', '2016-12-31') \
        .map(_mask_l8_sr)
dataset = dataset.reduce('median')
task = ee.batch.Export.image.toDrive(image=dataset.clip(region),
                                     description="test",
                                     folder="testing",
                                     region=region['coordinates'],
                                     scale=30,
                                     fileFormat='GeoTIFF',
                                     skipEmptyTiles=True)
task.start()

In [11]:
images = []

#x = [16, 17, 16]
#y = [46, 46, 90]


x = [17, 16]
y = [46, 25]

pts = []
for i in range(0, len(x)):

    p = ee.Feature(ee.Geometry.Point(x[i], y[i]))
    pts.append(p)

pts = ee.List(pts)
   
pts = ee.FeatureCollection(pts)
    
result = result_pixels(range(2000, 2004), l8sr)



AttributeError: 'dict' object has no attribute 'prepare_for_export'

In [404]:
len(result)
len(result[0])
len(result[0]['B1_median'])

result[0]['B1_median']
           
#[0.02835, 0.04235] [16, 17, 16] [46, 46, 90]
#[0.04235, 0.02835] [16, 17] [46, 46]
#[0.02835]  [17, 16] [46, 90]
#[0.02835, 0.109]  [17, 16] [46, 25]

[0.02835, 0.109]

In [415]:
all_years_res = []
print("Years 2000->2004")
all_years_res.append(result_pixels(range(2000, 2004), l8sr))

print("Years 2004->2008")
all_years_res.append(result_pixels(range(2004, 2008), l8sr))

print("Years 2008->2012")
all_years_res.append(result_pixels(range(2008, 2012), l8sr))

print("Years 2012->2016")
all_years_res.append(result_pixels(range(2012, 2016), l8sr))

print("Years 2016->2018")
all_years_res.append(result_pixels(range(2016, 2019), l8sr))

Years 2000->2004


AttributeError: 'dict' object has no attribute 'prepare_for_export'

In [374]:
print(len(all_years_res))
print(len(all_years_res[0]))
print(len(all_years_res[0][0]))
print(len(all_years_res[0][0]['B1_median']))

5
4
7
1083


In [375]:
# make one year
nmb_turbines = len(all_years_res[0][0]['B1_median'])
b1_median = np.zeros((nmb_turbines, 19))

length_slots = 0
for i in range(0, len(all_years_res)):
    length_this = len(all_years_res[i])
    print(i)
    print(length_this)
    for j in range(0, length_this):
         print(len(all_years_res[i][j]['B1_median']))
         #b1_median[:, length_slots + j] = all_years_res[i][j]['B1_median']


0
4
1083
1083
1083
1083
1
4
1083
1082
1083
1083
2
4
1083
1083
1083
1083
3
4
1081
1083
1083
1065
4
3
1083
1083
1083


In [224]:
from time import time

start_time_all = time()

result_list = []




for year in range(0, len(years)):
#for year in range(0, 1):
    
    start_time = time()
    
    print(f'Current year: {year}')
    
    one_year_data = one_year(year)
    result_list.append(one_year_data)
    
    end_time = time()
    
    print(f'One year took {end_time - start_time} seconds')
    
end_time_all = time()
print("All took " + str(end_time_all - start_time_all) + " seconds")
    

Current year: 0
One year took 16.28039002418518 seconds
Current year: 1
One year took 118.34487628936768 seconds
Current year: 2


KeyboardInterrupt: 

In [216]:
# 0: 600: 1 lacking entry
# 0: 300: 0 lacking entries
# 301: 600: 0 lacking entries
# 0: 500: 
result_list[0].shape

(1083, 7)

In [137]:
np_results = np.vstack(result_list)


    
    

In [139]:
#np_results.reshape((1082, len(years), len(bands_select)))

np_results.reshape((1082, 1, len(bands_select)))

#np_results.reshape((10, len(years, len(bands_select))))

array([[[0.03385, 0.05055, 0.05715, ..., 0.1554 , 0.2796 , 0.0852 ]],

       [[0.0168 , 0.0265 , 0.0246 , ..., 0.05055, 0.2793 , 0.0233 ]],

       [[0.017  , 0.0218 , 0.01555, ..., 0.03005, 0.2784 , 0.0121 ]],

       ...,

       [[0.06205, 0.08505, 0.07605, ..., 0.18275, 0.2924 , 0.10025]],

       [[0.0553 , 0.0816 , 0.08695, ..., 0.2078 , 0.297  , 0.1345 ]],

       [[0.0673 , 0.10055, 0.1152 , ..., 0.2372 , 0.2934 , 0.1656 ]]])

In [122]:
len(years)

19

In [120]:
20536 / 19

1080.842105263158

In [ ]:
COUNTRY = 'AT'

file_years = fpr.get_param(
    COUNTRY, "PATH_TURBINE_LOCATIONS") + "year_assessment_30_7bands.txt"

# convert to numpy array
np.savetxt(file_years, all_results)

In [ ]:
data = np.loadtxt(file_years)

n_ready = sum(data[:, 0] > 0) / 7

print(f'nmb of turbines checked {n_ready}')


In [ ]:
turbines = pd.read_csv(fpr.get_param(COUNTRY, "FILE_TURBINE_LOCATIONS"))

print(turbines.shape)

idx = turbines['Jahr'] > 1999

turbines = turbines[idx]

print(turbines.shape)

n = turbines.shape[0]
results = np.zeros((n, 2))

len_data = round(data.shape[0] / 7)

data_reshaped = data.reshape((len_data, 7, 19))


In [ ]:
data_reshaped

In [ ]:
validation = pd.DataFrame(np.argmax(data_reshaped, axis = 2) + 2000)
validation['Observed'] = turbines['Jahr'].values

#idx = (validation['Observed'] > 2008) & (validation['Observed'] < 2016)

#validation = validation[idx]

#print(validation.shape)

#n_ready_real = validation.shape[0]

validation.head(1000)

In [ ]:
#plt.scatter(validation['Estimate'], validation['Observed'])
for i in range(0, 7):
    plt.hist(validation[i] - validation['Observed'])
    diff = validation[i] - validation['Observed']
    #print(diff.head())
    plt.figure()
    #plt.plot(range(0, 19), data[3, ])
    
    #validation.iloc[3, :]

In [ ]:
validation['Estimate'] = validation['Estimate']

dev_0 = sum((validation['Estimate'] - validation['Observed']) == 0)
dev_1 = sum((validation['Estimate'] - validation['Observed']) == 1)
dev_m1 = sum((validation['Estimate'] - validation['Observed']) == -1)
dev_2 = sum((validation['Estimate'] - validation['Observed']) == 1)
dev_m2 = sum((validation['Estimate'] - validation['Observed']) == -1)

print(dev_0)
print(dev_1)
print(dev_m1)
print(dev_2)
print(dev_m2)

In [ ]:
data_diff = data[:, 1:19] - data[:, 0:18]

validation = pd.DataFrame({'Estimate' : np.argmax(data_diff, axis=1) + 2000, 
              'Observed' : turbines['Jahr']}, 
             columns = ['Estimate', 'Observed'])

idx = validation['Observed'] > 2004

validation = validation[idx]

n_ready_real = validation.shape[0]

dev_0 = sum((validation['Estimate'] - validation['Observed']) == 0)
dev_1 = sum((validation['Estimate'] - validation['Observed']) == 1)
dev_m1 = sum((validation['Estimate'] - validation['Observed']) == -1)
dev_2 = sum((validation['Estimate'] - validation['Observed']) == 1)
dev_m2 = sum((validation['Estimate'] - validation['Observed']) == -1)

print(dev_0)
print(dev_1)
print(dev_m1)
print(dev_2)
print(dev_m2)

print((dev_0 + dev_1 + dev_m1 + dev_2 + dev_m2) / n_ready_real)